In [1]:
#pip install tqdm

In [2]:
import pandas as pd
import numpy as np
import tabula
import re 
import sqlite3
from tqdm import tqdm


C:\Users\Usuario\AppData\Local\Temp\ipykernel_2828\1722563103.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
def remove_accents_and_replace(text):
    text = text.strip()
    text = text.lower()
    text = text.replace("(-) ", "")
    text = (
        text.replace("á", "a")
        .replace("é", "e")
        .replace("í", "i")
        .replace("ó", "o")
        .replace("ú", "u")
    )
    text = text.replace("año", "anio")
    text = text.replace("ü", "u").replace("ñ", "n").replace("ç", "c")
    text = text.replace(" ", "_")
    text = re.sub(r"\W+", "", text)
    return text

In [4]:
def rename_duplicate_columns(df):
    cols = pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique():
        cols[cols[cols == dup].index.values.tolist()] = [
            dup + "_" + str(i) if i != 0 else dup for i in range(sum(cols == dup))
        ]
    df.columns = cols
    return df

In [5]:
files = [
    "../Inputs/DocEconomica_384_3_1_1_2020_12_31_00_00_00_000.pdf",
    "../Inputs/DocEconomica_384_3_1_1_2021_12_31_00_00_00_000.pdf",
    "../Inputs/DocEconomica_384_3_1_1_2022_12_31_00_00_00_000.pdf",
    "../Inputs/DocEconomica_6712_3_1_1_2022_12_31_00_00_00_000.pdf",
    "../Inputs/DocEconomica_736008_3_1_1_2021_12_31_00_00_00_000.pdf",
    "../Inputs/DocEconomica_736008_3_1_1_2022_12_31_00_00_00_000.pdf",
]

In [6]:
dfPdf = pd.DataFrame()
#listaColumnas = []
filesLength = len(files)
try:
    for document in files:
        dfPdf = tabula.read_pdf(document, pages="all", multiple_tables = True,
            lattice=True, output_format="dataframe",encoding="latin-1")[0]
        
        listaColumnas = dfPdf.columns.to_list()

        rowStartIndex = dfPdf.index[ dfPdf[listaColumnas[0]]=="ESTADO DE SITUACIÓN FINANCIERA" ].to_list()
        dfHeader = dfPdf.iloc[ :rowStartIndex[0] ].copy()

        dfPdf.drop([listaColumnas[1]], axis=1, inplace=True)
        dfPdf.columns = ["CUENTA","VALOR EN USD"]

        listaColumnas.clear()
        listaColumnas = dfPdf.columns.to_list()

        indexCodValor = dfPdf[ (dfPdf[listaColumnas[0]]==listaColumnas[0]) &
                        (dfPdf[listaColumnas[1]]=="VALOR (En USD$)")  
                        ].index
    
        dfPdf.drop(indexCodValor, inplace=True)

        patternDelNull = r"ull"
        filterNullStrings = dfPdf[ dfPdf[listaColumnas[0]].str.contains(patternDelNull, na=False) ].index
        dfPdf.drop(filterNullStrings, inplace=True)

        filterNullStrings = dfPdf[ (dfPdf[listaColumnas[0]].isna()) &
                                (dfPdf[listaColumnas[1]].isna()) ].index
        dfPdf.drop(filterNullStrings, inplace=True)

        dfPdf.reset_index(drop=True, inplace=True)

        rowStartIndex = dfPdf.index[ dfPdf[listaColumnas[0]]=="REPRESENTANTE(S) LEGAL(ES)"  ].to_list()
        rowEndIndex =  dfPdf.index[ dfPdf[listaColumnas[0]]=="CONTADOR"  ].to_list() 

        dfContador = dfPdf.iloc[ rowStartIndex[0]: ].copy()
        dfContador.columns = ["REPRESENTANTE(S) LEGAL(ES)","CONTADOR"]

        filterNullStrings = dfContador[ 
                    (dfContador[dfContador.columns.to_list()[0]]=="REPRESENTANTE(S) LEGAL(ES)") | 
                    (dfContador[dfContador.columns.to_list()[0]]=="CONTADOR")
                    ].index
        dfContador.drop(filterNullStrings, inplace=True)
        
        listaColumnasContador = dfContador.columns.to_list()
        dfContador.reset_index(drop=True, inplace=True)

        pattern = dfContador.loc[0,"REPRESENTANTE(S) LEGAL(ES)"].split("\r")[0]
        similar_rows = dfContador[dfContador[listaColumnasContador[0]].str.contains(pattern, case=False, na=False)]
        if(len(similar_rows) > 1):
            dfContador.drop(similar_rows.index.to_list()[0], inplace=True)
        
        dfContador.reset_index(drop=True, inplace=True)
        ColumnasContador= ["REPRESENTANTE LEGAL","IDENT. REPRESENTANTE LEGAL","CONTADOR","IDENT. CONTADOR","COD.CONTADOR"]

        datosContador = []
        if len(dfContador) >= len(ColumnasContador):
            for k in range(len(ColumnasContador)):
                datosContador.append( dfContador.loc[k,listaColumnasContador[0]] )
        else:
            for k in range(len(ColumnasContador)):
                if k in range(len(dfContador)):
                    datosContador.append( dfContador.loc[k,listaColumnasContador[0]] )
                else:
                    datosContador.append( " " )

        headerColumns = dfHeader.columns.to_list()
        dfHeader.drop([headerColumns[-1]], axis=1, inplace=True)
        headerColumns.pop(0)

        dfHeader.columns = headerColumns
        newFirstRow = pd.DataFrame([dfHeader.columns.to_list()], columns=dfHeader.columns)
        dataFrameHeaderContador = pd.concat([newFirstRow, dfHeader])
        dataFrameHeaderContador.reset_index(drop=True, inplace=True)

        rowStartIndex = dfPdf.index[ dfPdf[listaColumnas[0]]=="ESTADO DE SITUACIÓN FINANCIERA" ].to_list()
        dfPdf = dfPdf.iloc[ rowStartIndex[0]: ]
        dfPdf.reset_index(drop=True, inplace=True)

        rowStartIndex = dfPdf.index[ dfPdf[listaColumnas[0]]=="REPRESENTANTE(S) LEGAL(ES)" ].to_list()
        dfPdf = dfPdf.iloc[:rowStartIndex[0]]

        filterNullStrings = dfPdf[  (dfPdf[listaColumnas[0]]=="ESTADO DE SITUACIÓN FINANCIERA") &
                        (dfPdf[listaColumnas[1]].isna()) ].index
        dfPdf.drop(filterNullStrings, inplace=True)

        filterNullStrings = dfPdf[  (dfPdf[listaColumnas[0]].isna()) &
                        (dfPdf[listaColumnas[1]].isna()) ].index
        dfPdf.drop(filterNullStrings, inplace=True)
        dfPdf.reset_index(drop=True, inplace=True)

        if len(datosContador) <= 0:
            for k in range(len(ColumnasContador)):
                datosContador.append(" ")

        dfContador = pd.DataFrame([datosContador],columns = ColumnasContador)
        transposedHeader = dataFrameHeaderContador.set_index( dataFrameHeaderContador.columns.to_list()[0]).T

        for k in range( len(dfContador.columns.to_list()) ):
            transposedHeader.insert( k,dfContador.columns.to_list()[k], dfContador.loc[0, dfContador.columns.to_list()[k]] )

        transposedHeader.reset_index(drop=True, inplace=True)

        transposedData = dfPdf.set_index(listaColumnas[0]).T

        transposedData.columns = [
            remove_accents_and_replace(col) for col in transposedData.columns
        ]
        datosTransposed = rename_duplicate_columns(transposedData)



        conn = sqlite3.connect("../Outputs/datos_pdf.db")
        cursor = conn.cursor()

        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='contabilidad'")
        table_exists = cursor.fetchone()

        if not table_exists:
            cursor.execute("CREATE TABLE contabilidad (id INTEGER PRIMARY KEY)")

        cursor.execute("PRAGMA table_info(contabilidad)")
        existing_columns = [column[1] for column in cursor.fetchall()]

        new_columns = [column for column in datosTransposed.columns if column not in existing_columns]

        for column in new_columns:
            cursor.execute(f"ALTER TABLE contabilidad ADD COLUMN {column} NULL")         

        conn.commit()

        datosTransposed.to_sql("contabilidad", conn, if_exists="append", index=False)
        transposedHeader.to_sql("cabecera", conn, if_exists="append", index=False)
        conn.close()
except Exception as e:
    print(f"error: {e}")



Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'


In [7]:
#conn = sqlite3.connect("../Outputs/datos3.db")
#cursor = conn.cursor()

#cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='contabilidad'")
#table_exists = cursor.fetchone()

#if not table_exists:
#    cursor.execute("CREATE TABLE contabilidad (id INTEGER PRIMARY KEY)")

#cursor.execute("PRAGMA table_info(contabilidad)")
#existing_columns = [column[1] for column in cursor.fetchall()]

#new_columns = [column for column in datosTransposed.columns if column not in existing_columns] 

#for column in new_columns:
#    cursor.execute(f"ALTER TABLE contabilidad ADD COLUMN {column} NULL")

#conn.commit()

#datosTransposed.to_sql("contabilidad", conn, if_exists="append", index=False)
#transposedHeader.to_sql("cabecera", conn, if_exists="append", index=False)
#conn.close()